In [ ]:
import glob, os
DATA_FOLDER="/n/sd7/trung/csp/data/erica/dialogue/data-kyoto-1711/elan_tab"
OUTPUT_FOLDER="/n/sd7/trung/data/erica"
from subprocess import call
from struct import unpack, pack
import numpy as np

import IPython
import wave
from pydub import AudioSegment

In [33]:
words = [s.strip().split(' ', 1) for s in open('/n/rd32/mimura/e2e/data/script/aps_sps/word.id', encoding='eucjp')]
decoder_map_word = {word[0].split('+')[0]: int(word[1]) for word in words}

In [34]:
filepaths = []
targets = []

for file in glob.glob(os.path.join(DATA_FOLDER, "20171128_*.txt")):
    lines = open(file).read().split('\n')
    lines = [line.split('\t') for line in lines]
    lines = list(filter(lambda line: line[0] == "Utterance(ERICA)", lines))
    lines = list(map(lambda line: (float(line[1]) * 1000, float(line[2]) * 1000, line[3]), lines))
    
    id = os.path.basename(file).split('.')[0]
    
    wavpath = '/n/sd7/trung/csp/data/erica/dialogue/data-kyoto-1711/20171128/%s/%s_audio_mix.wav' % (id, id)
    
    wav = AudioSegment.from_wav(wavpath)
    
    for count, (start, end, text) in enumerate(lines):
        targets.append(text)
        if not os.path.exists(os.path.join(OUTPUT_FOLDER, "wav", id)):
            os.mkdir(os.path.join(OUTPUT_FOLDER, "wav", id))
        if not os.path.exists(os.path.join(OUTPUT_FOLDER, "htk", id)):
            os.mkdir(os.path.join(OUTPUT_FOLDER, "htk", id))
        #wav[start:end].export(os.path.join(OUTPUT_FOLDER, "wav", id, "%06d.wav" % (count + 1)), format='wav')
        '''
        call([
            "/n/sd7/trung/bin/htk/HTKTools/HCopy",
            os.path.join(OUTPUT_FOLDER, "wav", id, "%06d.wav" % (count + 1)),
            os.path.join(OUTPUT_FOLDER, "htk", id, "%06d.htk" % (count + 1)),
            "-C", "/n/sd7/trung/config.lmfb.40ch"
        ])
        '''
        filepaths.append(os.path.join(OUTPUT_FOLDER, "htk", id, "%06d.htk" % (count + 1)))

print(len(filepaths))
with open('/n/sd7/trung/data/erica/inputs.txt', 'w') as f:
    f.write('\n'.join(filepaths))
    
with open('/n/sd7/trung/data/erica/targets.txt', 'w') as f:
    f.write('\n'.join(targets))

1540


In [35]:
wav[test[0]:test[1]].export("/n/sd7/trung/test.wav", format='wav')

NameError: name 'test' is not defined

In [ ]:
IPython.display.Audio("/n/sd7/trung/test.wav", autoplay=True)

In [78]:
mean = open("/n/sd7/trung/data/aps_sps_mean.dat").read().split('\n')[:-1]
mean = np.array([float(x) for x in mean])
var = open("/n/sd7/trung/data/aps_sps_var.dat").read().split('\n')[:-1]
var = np.array([float(x) for x in var])

def get_features(infile):
    outfile = "/n/sd7/trung/tmp/tmp.htk"
    call([
        "/n/sd7/trung/bin/htk/HTKTools/HCopy",
        infile, outfile,
        "-C", "/n/sd7/trung/config.lmfb.40ch", "-T", "1"
    ])
    fh = open(outfile, "rb")
    spam = fh.read(12)
    nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
    veclen = int(sampSize / 4)
    fh.seek(12, 0)
    dat = np.fromfile(fh, dtype=np.float32)
    dat = dat.reshape(len(dat) // veclen, veclen)
    dat = dat.byteswap()
    fh.close()
    
    dat = (dat - mean) / var
    fh = open(outfile, "wb")
    fh.write(spam)
    fh.seek(12, 0)
    dat.tofile(fh)
    fh.close()
    
    return dat

In [79]:
wavfile = "/n/sd7/trung/data/erica/wav/20171128_01/000026.wav"
feat = get_features(wavfile)
print(feat)

[[-1.3563083  -0.60438748 -0.16516968 ...  1.63716888  2.33180245
   2.6821463 ]
 [-0.93287117 -0.96212057 -0.44866459 ... -0.39365604 -1.65758156
  -2.74400562]
 [-0.72287086 -1.13310691 -0.50432257 ... -2.02688317 -4.71424949
  -8.23852352]
 ...
 [-0.92506815 -0.50430951 -0.11062722 ... -0.31870232  4.21929527
   5.22132578]
 [-0.56908713 -0.61964139 -0.26090718 ... -0.88665003  2.31265971
   3.53524767]
 [-0.7744945  -0.8890716  -0.54675646 ... -0.70062965  0.19037309
   1.76059851]]


In [64]:
IPython.display.Audio(wavfile, autoplay=True)

[5.513117426324923, 6.4372726280537265, 7.039052108515318, 7.483459436966893, 7.609096689740643, 7.597438146612656, 7.727616169758449, 7.85137872933976, 7.882184887914067, 7.869159582939171, 7.714500598951592, 7.694548865529827, 7.693034916473866, 7.688998289856246, 7.727658937707531, 7.740299232991686, 7.812012751839686, 7.900521260480068, 8.0075131502701, 8.09459954112279, 8.160358509597945, 8.222066914241589, 8.367843925996995, 8.560241463637627, 8.659760535805164, 8.619555790360971, 8.592388269891694, 8.660849707278981, 8.761198226934408, 8.831993787539389, 8.838446808434268, 8.795840819718494, 8.744237061817671, 8.721504993000874, 8.742108580329376, 8.768381239430582, 8.81139461277047, 8.878832216751151, 8.896731927018982, 8.270544314055897, 0.000430728709860992, 0.0005279870220909271, 0.000674947339800168, 0.0007102158103887029, 0.0006458182483911633, 0.0006222750711565015, 0.0006465713085563652, 0.0006420961103828666, 0.0006068078268232923, 0.0006063555898882477, 0.0006057679879

5.513117426324923
